In [1]:
import matplotlib.pyplot as plt
from sklearn.svm import SVC 
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
from os import path

In [2]:
df = pd.read_csv("cumulative.csv")

In [3]:
df.head()

,rowid,kepid,kepoi_name,kepler_name,koi_disposition,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,1,10797460,K00752.01,Kepler-227 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,2,10797460,K00752.02,Kepler-227 c,CONFIRMED,CANDIDATE,0.969,0,0,0,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,3,10811496,K00753.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,4,10848459,K00754.01,NaN,FALSE POSITIVE,FALSE POSITIVE,0.000,0,1,0,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,5,10854555,K00755.01,Kepler-664 b,CONFIRMED,CANDIDATE,1.000,0,0,0,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [4]:
drop_columns = ['rowid', 'kepid', 'kepoi_name', 'kepler_name', 'koi_disposition','koi_tce_delivname']

In [5]:
df = df.drop(columns = drop_columns)

In [6]:
df = df.dropna(axis = "columns", how = "all")

In [7]:
df = df.dropna()

In [8]:
df["koi_pdisposition"].value_counts()

CANDIDATE         3996
FALSE POSITIVE    3807
Name: koi_pdisposition, dtype: int64

In [9]:
df.columns

Index(['koi_pdisposition', 'koi_score', 'koi_fpflag_nt', 'koi_fpflag_ss',
       'koi_fpflag_co', 'koi_fpflag_ec', 'koi_period', 'koi_period_err1',
       'koi_period_err2', 'koi_time0bk', 'koi_time0bk_err1',
       'koi_time0bk_err2', 'koi_impact', 'koi_impact_err1', 'koi_impact_err2',
       'koi_duration', 'koi_duration_err1', 'koi_duration_err2', 'koi_depth',
       'koi_depth_err1', 'koi_depth_err2', 'koi_prad', 'koi_prad_err1',
       'koi_prad_err2', 'koi_teq', 'koi_insol', 'koi_insol_err1',
       'koi_insol_err2', 'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff',
       'koi_steff_err1', 'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1',
       'koi_slogg_err2', 'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra',
       'dec', 'koi_kepmag'],
      dtype='object')

In [10]:
df.head()

,koi_pdisposition,koi_score,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CANDIDATE,1.000,0,0,0,0,9.488036,2.775000e-05,-2.775000e-05,170.538750,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,CANDIDATE,0.969,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,...,-81.0,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
2,FALSE POSITIVE,0.000,0,1,0,0,19.899140,1.494000e-05,-1.494000e-05,175.850252,...,-176.0,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
3,FALSE POSITIVE,0.000,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,...,-174.0,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
4,CANDIDATE,1.000,0,0,0,0,2.525592,3.761000e-06,-3.761000e-06,171.595550,...,-211.0,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509


In [11]:
# Assign X (data) and y (target)
X = df.drop("koi_pdisposition", axis=1)
y = df["koi_pdisposition"]
print(X.shape, y.shape)

(7803, 41) (7803,)


In [12]:
#Split our data into training and testing
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

## MinMax Scaler

In [13]:
#Fit on the training data
X_scaler = MinMaxScaler().fit(X_train)
X_scaler.get_params()

{'clip': False, 'copy': True, 'feature_range': (0, 1)}

In [14]:
#Fit on the training data
# X_scaler = StandardScaler().fit(X_train)
# X_scaler.get_params()

In [15]:
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)
X_test_scaled

array([[0.999     , 0.        , 0.        , ..., 0.66362782, 0.84637293,
        0.81153254],
       [0.997     , 0.        , 0.        , ..., 0.40754724, 0.62296282,
        0.78872092],
       [0.        , 0.        , 0.        , ..., 0.36489169, 0.16733163,
        0.61473703],
       ...,
       [0.146     , 1.        , 0.        , ..., 0.68831833, 0.43179568,
        0.81117045],
       [0.943     , 0.        , 0.        , ..., 0.29930692, 0.75797076,
        0.7740563 ],
       [0.        , 0.        , 0.        , ..., 0.56064804, 0.70768092,
        0.78238436]])

In [16]:
y_train.value_counts()

CANDIDATE         2990
FALSE POSITIVE    2862
Name: koi_pdisposition, dtype: int64

In [17]:
y_test.value_counts()

CANDIDATE         1006
FALSE POSITIVE     945
Name: koi_pdisposition, dtype: int64

In [18]:
# Create the SVC Model
model = SVC(kernel="rbf")
model

SVC()

# predictions without grid search

In [19]:
model.fit(X_train, y_train)

SVC()

In [20]:
print('Test Acc: %.3f' % model.score(X_test, y_test))

Test Acc: 0.698


In [21]:
# Calculate classification report
from sklearn.metrics import classification_report
predictions = model.predict(X_test)
print(classification_report(y_test, predictions,
                            target_names=["candidate", "false positive"]))

                precision    recall  f1-score   support

     candidate       0.64      0.97      0.77      1006
false positive       0.93      0.41      0.57       945

      accuracy                           0.70      1951
     macro avg       0.78      0.69      0.67      1951
  weighted avg       0.78      0.70      0.67      1951



## Grid Search

In [22]:
# Create the GridSearch estimator along with a parameter object containing the values to adjust
param_grid = {'C': [1, 5, 10, 50],
              'gamma': [0.0001, 0.0005, 0.001, 0.005]}
grid = GridSearchCV(model, param_grid, verbose=3)

In [23]:
# Fit the model using the grid search estimator. 
# This will take the SVC model and try each combination of parameters
grid.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 16 candidates, totalling 80 fits
[CV 1/5] END ..............................C=1, gamma=0.0001; total time=   2.5s
[CV 2/5] END ..............................C=1, gamma=0.0001; total time=   2.5s
[CV 3/5] END ..............................C=1, gamma=0.0001; total time=   2.4s
[CV 4/5] END ..............................C=1, gamma=0.0001; total time=   2.3s
[CV 5/5] END ..............................C=1, gamma=0.0001; total time=   2.3s
[CV 1/5] END ..............................C=1, gamma=0.0005; total time=   1.5s
[CV 2/5] END ..............................C=1, gamma=0.0005; total time=   1.5s
[CV 3/5] END ..............................C=1, gamma=0.0005; total time=   1.5s
[CV 4/5] END ..............................C=1, gamma=0.0005; total time=   1.5s
[CV 5/5] END ..............................C=1, gamma=0.0005; total time=   1.5s
[CV 1/5] END ...............................C=1, gamma=0.001; total time=   0.8s
[CV 2/5] END ...............................C=1,

GridSearchCV(estimator=SVC(),
             param_grid={'C': [1, 5, 10, 50],
                         'gamma': [0.0001, 0.0005, 0.001, 0.005]},
             verbose=3)

In [24]:
# List the best parameters for this dataset
print(grid.best_params_)

{'C': 50, 'gamma': 0.005}


In [25]:
# List the best score
print(grid.best_score_)

0.9989747969081872


In [26]:
# Make predictions with the hypertuned model
predictions = grid.predict(X_test)

In [27]:
print('Test Acc: %.3f' % grid.score(X_test, y_test))

Test Acc: 0.484


In [28]:
predictions

array(['FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE', ...,
       'FALSE POSITIVE', 'FALSE POSITIVE', 'FALSE POSITIVE'], dtype=object)

In [29]:
# Calculate classification report
from sklearn.metrics import classification_report
print(classification_report(y_test, predictions,
                            target_names=["candidate", "false positive"]))

                precision    recall  f1-score   support

     candidate       0.00      0.00      0.00      1006
false positive       0.48      1.00      0.65       945

      accuracy                           0.48      1951
     macro avg       0.24      0.50      0.33      1951
  weighted avg       0.23      0.48      0.32      1951



C:\Users\yenia\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yenia\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Users\yenia\anaconda3\envs\PythonAdv\lib\site-packages\sklearn\metrics\_classification.py:1245: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
